In [1]:
# import tensorflow as tf
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
# from tensorflow.keras.preprocessing import image
# import numpy as np
# import os
# import shutil


# # Load the MobileNetV2 model pre-trained on ImageNet
# model = MobileNetV2(weights='imagenet')


2024-05-31 10:50:30.270328: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 10:50:31.021146: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/iv/miniconda3/envs/tf/lib/
2024-05-31 10:50:31.021246: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/iv/miniconda3/envs/tf/lib/
2024-05-31 10:50:31.021256: W tensorflow/compiler/tf2tensorrt/

In [1]:
import tensorflow as tf
# from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.resnet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import os
import shutil

from tensorflow.keras.applications import ResNet50
model = ResNet50(weights='imagenet')

2024-05-31 10:51:56.721062: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 10:51:57.441100: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/iv/miniconda3/envs/tf/lib/
2024-05-31 10:51:57.441184: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/iv/miniconda3/envs/tf/lib/
2024-05-31 10:51:57.441192: W tensorflow/compiler/tf2tensorrt/

In [2]:
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array


In [3]:
def classify_and_sort_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for img_name in os.listdir(input_folder):
        img_path = os.path.join(input_folder, img_name)
        # print(img_name)
        file_type = img_name.split('.')[-1]
        if file_type not in ['jpg','jpeg','png']:
            continue
        img_array = preprocess_image(img_path)

        # Predict the class of the image
        predictions = model.predict(img_array)
        decoded_predictions = decode_predictions(predictions, top=1)[0]
        predicted_class = decoded_predictions[0][1]

        # Create a directory for the predicted class if it doesn't exist
        class_folder = os.path.join(output_folder, predicted_class)
        if not os.path.exists(class_folder):
            os.makedirs(class_folder)

        # Move the image to the class folder
        shutil.move(img_path, os.path.join(class_folder, img_name))




In [4]:
# Example usage
input_folder = 'img_input'
output_folder = 'sorted_images'
classify_and_sort_images(input_folder, output_folder)

2024-05-31 10:52:55.606964: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2024-05-31 10:52:55.962343: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-05-31 10:52:56.005472: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-05-31 10:52:56.005522: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:85] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-05-31 10:52:56.048972: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-05-31 10:52:56.049073: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


1/1 [==============================] - 0s 25ms/step
